# Spin Dynamics Workshop — LLG Simulations with UppASD Python Interface

By the end of this notebook you will have run **realistic spin-dynamics simulations** using the **UppASD framework** that:
- reads and builds lattice and interaction structures,
- initializes magnetic configurations (random, uniform, skyrmions, etc.),
- integrates the Landau-Lifshitz-Gilbert (LLG) equation with physics-validated solvers,
- visualizes and analyzes energy, magnetization, and spin trajectories interactively.

We maintain the **didactic flow** of the original workshop, but now the core mechanics and simulation tasks are carried by **UppASD's Python interface**, which wraps the Fortran engine. This ensures:
- **Accuracy**: Physics computed by the peer-reviewed Fortran code
- **Reproducibility**: Files are the contract (inpsd.dat, posfile, momfile, jfile, dmfile)
- **Educational clarity**: Build systems step-by-step, understand each layer

---

## Setup: Install and Import UppASD

First, ensure UppASD is installed. If not already available, we install it with wheels.

In [ ]:
import importlib.util
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import trange

# Install UppASD if not available
if importlib.util.find_spec("uppasd") is None:
    print("Installing uppasd…")
    !pip install --pre --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple uppasd

# Import UppASD core classes
from uppasd.core.system import SpinSystem
from uppasd.core.exchange import ExchangeShellTable, DMIShellTable
from uppasd.input.inputdata import ASDInput
from uppasd.run.simulator import ASDWorkspace, UppASDSimulator
from uppasd.core.results import ASDResults

print("✅ UppASD imported successfully")

## Enable Interactive Plotting

We enable interactive Matplotlib (ipympl) for 3D spin visualizations.

In [ ]:
try:
    import ipympl  # noqa: F401
    get_ipython().run_line_magic("matplotlib", "widget")
    print("Matplotlib backend: widget (ipympl) ✅")
except Exception as e:
    print("ipympl not available -> falling back to inline ❌")
    get_ipython().run_line_magic("matplotlib", "inline")

# Example 01 — Heisenberg Chain (1D, Periodic Boundary Conditions)

## Overview

We simulate a **1D nanochain** with:
- **Geometry**: 10 atoms in a linear chain with periodic boundary conditions (PBC)
- **Interactions**: 
  - Nearest-neighbor **Heisenberg exchange** ($J > 0$ ferromagnetic)
  - Nearest-neighbor **DMI** (Dzyaloshinskii-Moriya interaction) along $\hat{z}$
- **Dynamics**: LLG integration with damping to reach a steady state

### Physics

The magnetic Hamiltonian is:
$$H = -\sum_{i<j} J_{ij}\,\mathbf{S}_i\cdot\mathbf{S}_j\;-\;\sum_{i<j}\mathbf{D}_{ij}\cdot(\mathbf{S}_i\times\mathbf{S}_j)$$

The LLG equation of motion is:
$$\frac{d\mathbf{S}_i}{dt} = -\gamma\,\mathbf{S}_i\times\mathbf{H}^{\mathrm{eff}}_i\;-\;\alpha\gamma\,\mathbf{S}_i\times\left(\mathbf{S}_i\times\mathbf{H}^{\mathrm{eff}}_i\right)$$

where $\gamma$ is the gyromagnetic ratio and $\alpha$ is the damping constant.

## Step 1: Define System Geometry

We create a **1D chain** with 10 atoms and spacing 1.0 (in code units).

In [ ]:
# Example 01 — 1D chain parameters
N_chain = 10  # Total number of atoms (via ncell repetition)
a = 1.0       # lattice constant

# Define UNIT CELL with 1 atom (UppASD will replicate it)
cell_ex01 = np.diag([a, a, a])

# Single atom at origin
positions_ex01 = np.array([[0.0, 0.0, 0.0]])
natom_ex01 = 1

# Species: all the same (species index 1)
species_ex01 = np.ones(natom_ex01, dtype=int)

# Initial moments: pointing in +z direction
moments_ex01 = np.array([[0.0, 0.0, 1.0]])

# Create the spin system
system_ex01 = SpinSystem(cell_ex01, positions_ex01, species_ex01, moments_ex01)

print(f"✅ Created 1D chain unit cell:")
print(f"   Unit cell size: {cell_ex01.diagonal()}")
print(f"   Atoms in unit cell: {natom_ex01}")
print(f"   Will be replicated to {N_chain} atoms via ncell=(10,1,1)")

## Step 2: Define Exchange Interactions

We add **nearest-neighbor Heisenberg exchange** and **DMI** using UppASD's `ExchangeShellTable` and `DMIShellTable`.

In [ ]:
# Exchange interactions (Heisenberg)
J = 1.0  # ferromagnetic
D = 0.1  # DMI strength

exchange_ex01 = ExchangeShellTable()
dmi_ex01 = DMIShellTable()

# For a single-atom unit cell, define bonds to nearest neighbors
# Bond vectors point to adjacent unit cells
# UppASD will replicate these across the entire system

# Forward bond: atom 1 to its neighbor in +x direction
bond_forward = np.array([a, 0.0, 0.0], float)
exchange_ex01.add_bond(1, 1, 1, bond_forward, J)
dmi_ex01.add_bond(1, 1, 1, bond_forward, np.array([0.0, 0.0, D], float))

# Backward bond: atom 1 to its neighbor in -x direction
bond_backward = np.array([-a, 0.0, 0.0], float)
exchange_ex01.add_bond(1, 1, 1, bond_backward, J)
dmi_ex01.add_bond(1, 1, 1, bond_backward, np.array([0.0, 0.0, -D], float))

print(f"✅ Added exchange and DMI interactions:")
print(f"   J = {J}, D = {D}")
print(f"   Bonds defined for unit cell (will be replicated)")

## Step 3: Configure Input Parameters and Run Simulation

We set up the LLG solver parameters via `ASDInput`, prepare a workspace, and run the simulation.

In [ ]:
# Configure the ASDInput parameters
inp_ex01 = ASDInput()

# System block
inp_ex01.block('system').set(
    simid='chain_ex01',
    ncell=(N_chain, 1, 1),     # Replicate unit cell N_chain times in x-direction
    cell=cell_ex01,
    bc=(0, 0, 0),              # vacuum boundaries
)

# Dynamics block: LLG integration
inp_ex01.block('dynamics').set(
    mode='S',                  # SD = spin dynamics (LLG)
    temp=0.0,                  # zero temperature (deterministic)
    nstep=10000,                # number of MD steps
    timestep=1e-15,             # time step (code units)
    damping=0.5,               # damping constant α
)

# Measurement block: specify what to save
inp_ex01.block('measurement').set(
    do_avrg='Y',              # compute averages
    plotenergy=10,            # plot energy every N steps
    tottraj_step=10,          # trajectory output every N steps
)

# Initialize with random order
inp_ex01.block('initialization').set(
    initmag=1,
)
# Create workspace and prepare files
workspace_ex01 = ASDWorkspace('./run_ex01_chain', clean=True)
workspace_ex01.prepare(system=system_ex01, inp=inp_ex01, exchange=exchange_ex01, dmi=dmi_ex01)

print("✅ Input configured and workspace prepared")
print("   Running LLG simulation...")

# Run the simulation
simulator_ex01 = UppASDSimulator(workspace_ex01)
sim_result_ex01 = simulator_ex01.run_all()

print("✅ Simulation completed!")

## Step 4: Load and Analyze Results

We extract and visualize the simulation output.

In [ ]:
# Load results from the workspace
results_ex01 = ASDResults('./run_ex01_chain')

# Extract energy and magnetization from output
# totenergy returns dict with keys: 'iter', 'tot', 'exc', 'ani', 'dm', etc.
energy_data_ex01 = results_ex01['totenergy']
time_exe01 = energy_data_ex01['iter']
energy_ex01 = energy_data_ex01['tot']

# averages returns dict with keys: 'iter', 'mx', 'my', 'mz', 'm', 'm_std'
avg_data_ex01 = results_ex01['averages']
mag_ex01 = np.column_stack([avg_data_ex01['mx'], avg_data_ex01['my'], avg_data_ex01['mz']])
time_exm01 = avg_data_ex01['iter']
print(avg_data_ex01.keys())
print(f"✅ Loaded results:")
print(f"   Energy shape: {energy_ex01.shape}")
print(f"   Time points: {len(time_exe01)}")
print(f"   Magnetization: {mag_ex01.shape}")

# Plot energy evolution
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(time_exe01, energy_ex01, linewidth=1.5)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Total Energy', fontsize=12)
plt.title('Example 01 — Energy vs Time', fontsize=13)
plt.grid(True, alpha=0.3)

# Plot magnetization components
plt.subplot(1, 2, 2)
plt.plot(time_exm01, avg_data_ex01['mx'], label='Mx', linewidth=1.5)
plt.plot(time_exm01, avg_data_ex01['my'], label='My', linewidth=1.5)
plt.plot(time_exm01, avg_data_ex01['mz'], label='Mz', linewidth=1.5)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Magnetization (per atom)', fontsize=12)
plt.title('Example 01 — Magnetization vs Time', fontsize=13)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()

plt.show()
print(f"Final magnetization:  ({mag_ex01[-1, 0]:.3f}, {mag_ex01[-1, 1]:.3f}, {mag_ex01[-1, 2]:.3f})")

print(f"Final total energy:   {energy_ex01[-1]:.4f}")
print(f"Initial total energy: {energy_ex01[0]:.4f}")

---

# Example 02 — Skyrmion on a Square Lattice (2D, Periodic BCs)

## Overview

We simulate a **2D square lattice** (20×20 atoms) with:
- **Geometry**: Square lattice with periodic boundary conditions
- **Interactions**:
  - Ferromagnetic **Heisenberg exchange** ($J > 0$)
  - **Interfacial DMI** (perpendicular to the plane): $\mathbf{D}_{ij} = D_z\,(\hat{z}\times\hat{r}_{ij})$
  - **Perpendicular magnetic field** $B_z > 0$
- **Initial conditions**: Random spins or seeded skyrmion texture

### Physics

Interfacial DMI stabilizes **skyrmions** (topologically protected spin vortices) in thin films. The twist length is set by:

$$\lambda \sim \frac{2\pi}{\arctan(D/J)}$$

We choose parameters such that $\lambda \approx 20$ lattice sites, allowing one skyrmion to fit in the 20×20 cell.

## Step 1: Create Square Lattice Geometry

In [ ]:
# Example 02 — Square lattice skyrmion
Nx_sq, Ny_sq = 20, 20  # System size (via ncell repetition)
a_sq = 1.0             # lattice constant

# Define UNIT CELL with 1 atom (UppASD will replicate it)
cell_sq = np.diag([a_sq, a_sq, a_sq])

# Single atom at origin
positions_sq = np.array([[0.0, 0.0, 0.0]])
natom_sq = 1

species_sq = np.ones(natom_sq, dtype=int)

# Initial moments: pointing in +z direction
moments_sq = np.array([[0.0, 0.0, 1.0]])

system_sq = SpinSystem(cell_sq, positions_sq, species_sq, moments_sq)

print(f"✅ Created square lattice unit cell:")
print(f"   Unit cell size: {cell_sq.diagonal()}")
print(f"   Atoms in unit cell: {natom_sq}")
print(f"   Will be replicated to {Nx_sq}×{Ny_sq} = {Nx_sq * Ny_sq} atoms via ncell=({Nx_sq},{Ny_sq},1)")

## Step 2: Add Interfacial DMI and Exchange for Skyrmion Stabilization

We use a **DMI vector** perpendicular to bond directions in the xy-plane: $\mathbf{D}_{ij} = D\,(\hat{z}\times\hat{r}_{ij})$.

In [ ]:
# Skyrmion parameters
J_sq = 1.0
D_over_J_target = float(np.tan(2 * np.pi / 20.0))  # ~0.325 for λ~20
D_sq = float(D_over_J_target * J_sq)
Bz_sq = float(0.06 * J_sq)

print(f"D/J ≈ {D_sq/J_sq:.3f} (target λ ≈ 20 sites)")

exchange_sq = ExchangeShellTable()
dmi_sq = DMIShellTable()

# For single-atom unit cell: define bonds to 4 nearest neighbors
# Bond directions and corresponding DMI vectors (interfacial DMI: D_ij ⊥ bond)
for dx, dy, Dx, Dy in [(1, 0, D_sq, 0), 
                        (-1, 0, -D_sq, 0), 
                        (0, 1, 0, D_sq), 
                        (0, -1, 0, -D_sq)]:
    # Add bond from atom 1 to itself in neighboring unit cell
    exchange_sq.add_bond(1, 1, 1, [dx, dy, 0], J_sq)
    dmi_sq.add_bond(1, 1, 1, [dx, dy, 0], [Dx, Dy, 0])

print(f"✅ Interfacial DMI configured (D = {D_sq:.3f}, Bz = {Bz_sq:.3f})")
print(f"   4 bonds defined for unit cell (will be replicated)")

## Step 3: Run Skyrmion Relaxation

We relax the system with a perpendicular magnetic field (which stabilizes the skyrmion texture).

In [ ]:
# Configure input: skyrmion lattice
inp_sq = ASDInput()

inp_sq.block('system').set(
    simid='skyrmion',
    ncell=(Nx_sq, Ny_sq, 1),   # Replicate unit cell into Nx×Ny lattice
    cell=cell_sq,
    bc=(1, 1, 0),              # PBC in xy, vacuum in z
)

inp_sq.block('dynamics').set(
    mode='S',
    temp=0.0,
    nstep=10000,
    timestep=1e-15,
    damping=0.1,
)

inp_sq.block('measurement').set(
    do_avrg='Y',
    plotenergy=10,
    tottraj_step=20,
)

inp_sq.block('external').set(
    hfield=(0,0,Bz_sq),
)

workspace_sq = ASDWorkspace('./run_ex02_skyrmion', clean=True)
workspace_sq.prepare(system=system_sq, inp=inp_sq, exchange=exchange_sq, dmi=dmi_sq)

print("✅ Running skyrmion relaxation...")
simulator_sq = UppASDSimulator(workspace_sq)
sim_result_sq = simulator_sq.run_all()

print("✅ Skyrmion simulation completed!")

## Step 4: Analyze and Visualize Skyrmion Results

We plot energy, magnetization, and a 2D top view of the skyrmion texture.

In [ ]:
# Load results
results_sq = ASDResults('./run_ex02_skyrmion')

# Extract energy and magnetization from output dictionaries
energy_data_sq = results_sq['totenergy']
time_sq = energy_data_sq['iter']
energy_sq = energy_data_sq['tot']

avg_data_sq = results_sq['averages']
time_mag = avg_data_sq['iter']
mag_sq = np.column_stack([avg_data_sq['mx'], avg_data_sq['my'], avg_data_sq['mz']])

print(f"Skyrmion simulation results:")
print(f"   Initial Mz: {avg_data_sq['mz'][0]:.3f}")
print(f"   Final Mz:   {avg_data_sq['mz'][-1]:.3f}")

# Plot energy and magnetization
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(time_sq, energy_sq, linewidth=1.5, color='steelblue')
axes[0].set_xlabel('Iteration', fontsize=11)
axes[0].set_ylabel('Total Energy', fontsize=11)
axes[0].set_title('Example 02 — Skyrmion: Energy vs Time', fontsize=12)
axes[0].grid(True, alpha=0.3)

axes[1].plot(time_mag, avg_data_sq['mx'], label='Mx', linewidth=1.5)
axes[1].plot(time_mag, avg_data_sq['my'], label='My', linewidth=1.5)
axes[1].plot(time_mag, avg_data_sq['mz'], label='Mz', linewidth=1.5, color='red')
axes[1].set_xlabel('Iteration', fontsize=11)
axes[1].set_ylabel('Magnetization (per atom)', fontsize=11)
axes[1].set_title('Example 02 — Skyrmion: Magnetization vs Time', fontsize=12)

axes[1].legend(fontsize=10)
plt.show()

axes[1].grid(True, alpha=0.3)
plt.tight_layout()

---

# Example 03 — Kagomé Lattice: Non-Collinear Magnetism with DMI

## Overview

We simulate a **Kagomé lattice** (corner-sharing triangles) with:
- **Geometry**: 3-site basis per unit cell, periodic boundary conditions
- **Interactions**:
  - **Antiferromagnetic exchange** ($J < 0$) — introduces frustration
  - **$D_z$ DMI** (out-of-plane) — selects chirality
- **Magnetic order**: Non-collinear ~120° coplanar pattern (due to frustration + DMI)

### Physics

The Kagomé lattice is **geometrically frustrated**: three spins on each triangle cannot all align/antialign simultaneously. With AFM exchange + DMI, this leads to a **120° coplanar spin structure** characterized by:
- NN angle ≈ 120°
- Chiral spin texture (helical or whirlpool-like)
- Gapless excitations (magnons)

## Step 1: Build the Kagomé Lattice

We construct a Kagomé lattice from a triangular Bravais lattice with a 3-site basis.

In [ ]:
# Kagomé lattice setup
nx_k, ny_k = 8, 8  # Number of unit cells to replicate
a_k = 1.0          # Lattice constant

# Kagomé basis vectors (triangular Bravais lattice)
a1_k = np.array([a_k, 0.0, 0.0], float)
a2_k = np.array([0.5*a_k, 0.5*np.sqrt(3.0)*a_k, 0.0], float)

# Define UNIT CELL with 3 atoms (Kagomé basis)
basis_k = np.array([
    [0.0, 0.0, 0.0],                              # Site 1
    [0.5*a_k, 0.0, 0.0],                          # Site 2
    [0.25*a_k, 0.25*np.sqrt(3.0)*a_k, 0.0],       # Site 3
], float)

natom_k = 3  # 3 atoms per unit cell

# Cell vectors for Kagomé unit cell
cell_k = np.zeros((3, 3), float)
cell_k[0] = a1_k
cell_k[1] = a2_k
cell_k[2, 2] = 5.0  # vacuum in z

# Species: all the same
species_k = np.ones(natom_k, dtype=int)

# Initial 120° coplanar pattern (one per sublattice)
base_120 = np.array([
    [1.0, 0.0, 0.0],
    [-0.5, np.sqrt(3)/2, 0.0],
    [-0.5, -np.sqrt(3)/2, 0.0],
], float)

moments_k = base_120.copy()

system_k = SpinSystem(cell_k, basis_k, species_k, moments_k)

print(f"✅ Created Kagomé unit cell:")
print(f"   Unit cell: 3 atoms at Kagomé positions")
print(f"   Cell vectors: a1={a1_k}, a2={a2_k}")
print(f"   Will be replicated to {nx_k}×{ny_k} = {3*nx_k*ny_k} atoms via ncell=({nx_k},{ny_k},1)")
print(f"   Initial 120° coplanar pattern")

## Step 2: Add Kagomé Nearest-Neighbor Interactions

Each Kagomé site has 4 nearest neighbors. We add AFM exchange and $D_z$ DMI.

In [ ]:
# Kagomé NN connectivity within and between unit cells
# Each atom index (1, 2, 3) connects to neighbors via lattice vectors
J_k = -1.0  # AFM
Dz_k = 0.35

exchange_k = ExchangeShellTable()
dmi_k = DMIShellTable()

# Define bonds for 3-atom unit cell
# Each Kagomé site has 4 nearest neighbors
# Within-cell bonds (3 bonds) + between-cell bonds (3 more per atom)

# Within unit cell bonds
exchange_k.add_bond(1, 2, 1, basis_k[1] - basis_k[0], J_k)  # 1-2
exchange_k.add_bond(2, 1, 1, basis_k[0] - basis_k[1], J_k)  # 2-1
exchange_k.add_bond(1, 3, 1, basis_k[2] - basis_k[0], J_k)  # 1-3
exchange_k.add_bond(3, 1, 1, basis_k[0] - basis_k[2], J_k)  # 3-1
exchange_k.add_bond(2, 3, 1, basis_k[2] - basis_k[1], J_k)  # 2-3
exchange_k.add_bond(3, 2, 1, basis_k[1] - basis_k[2], J_k)  # 3-2

# Between unit cells - Atom 1 to neighbors in adjacent cells
bond_vec = -a1_k + (basis_k[1] - basis_k[0])
exchange_k.add_bond(1, 2, 1, bond_vec, J_k)
dmi_k.add_bond(1, 2, 1, bond_vec, np.array([0.0, 0.0, -Dz_k], float))

bond_vec = -a2_k + (basis_k[2] - basis_k[0])
exchange_k.add_bond(1, 3, 1, bond_vec, J_k)
dmi_k.add_bond(1, 3, 1, bond_vec, np.array([0.0, 0.0, -Dz_k], float))

# Between unit cells - Atom 2 to neighbors in adjacent cells
bond_vec = a1_k + (basis_k[0] - basis_k[1])
exchange_k.add_bond(2, 1, 1, bond_vec, J_k)
dmi_k.add_bond(2, 1, 1, bond_vec, np.array([0.0, 0.0, Dz_k], float))

bond_vec = a1_k - a2_k + (basis_k[2] - basis_k[1])
exchange_k.add_bond(2, 3, 1, bond_vec, J_k)
dmi_k.add_bond(2, 3, 1, bond_vec, np.array([0.0, 0.0, -Dz_k], float))

# Between unit cells - Atom 3 to neighbors in adjacent cells
bond_vec = a2_k + (basis_k[0] - basis_k[2])
exchange_k.add_bond(3, 1, 1, bond_vec, J_k)
dmi_k.add_bond(3, 1, 1, bond_vec, np.array([0.0, 0.0, Dz_k], float))

bond_vec = -a1_k + a2_k + (basis_k[1] - basis_k[2])
exchange_k.add_bond(3, 2, 1, bond_vec, J_k)
dmi_k.add_bond(3, 2, 1, bond_vec, np.array([0.0, 0.0, Dz_k], float))

print(f"✅ Kagomé interactions added:")
print(f"   J = {J_k} (AFM)")
print(f"   Dz = {Dz_k}")
print(f"   Bonds defined for 3-atom unit cell (will be replicated)")

## Step 3: Run Kagomé Relaxation

Relax the system from the 120° seed to find the ground state configuration.

In [ ]:
# Configure input: Kagomé
inp_k = ASDInput()

inp_k.block('system').set(
    simid='kagome_afm_dmi',
    ncell=(nx_k, ny_k, 1),     # Replicate 3-atom unit cell nx×ny times
    cell=cell_k,
    bc=(1, 1, 0),              # PBC in xy
)

inp_k.block('dynamics').set(
    mode='S',
    temp=0.0,
    nstep=5000,
    timestep=1.0e-15,
    damping=0.01,  # Higher damping for relaxation
)

inp_k.block('measurement').set(
    do_avrg='Y',
    plotenergy=10,
    tottraj_step=50,
)

workspace_k = ASDWorkspace('./run_ex03_kagome', clean=True)
workspace_k.prepare(system=system_k, inp=inp_k, exchange=exchange_k, dmi=dmi_k)

print("✅ Running Kagomé relaxation...")
simulator_k = UppASDSimulator(workspace_k)
sim_result_k = simulator_k.run_all()

print("✅ Kagomé simulation completed!")

## Step 4: Verify the 120° Magnetic Order

We extract results and verify that the final configuration has nearest-neighbor angles clustered around 120°.

In [ ]:
# Load results
results_k = ASDResults('./run_ex03_kagome')

# Extract energy and magnetization from output dictionaries
energy_data_k = results_k['totenergy']
time_k = energy_data_k['iter']
energy_k = energy_data_k['tot']

avg_data_k = results_k['averages']
time_avg = avg_data_k['iter']
mag_k = np.column_stack([avg_data_k['mx'], avg_data_k['my'], avg_data_k['mz']])

print(f"Kagomé final state:")
print(f"   Final energy: {energy_k[-1]:.4f}")
print(f"   Final <M>: ({avg_data_k['mx'][-1]:.4f}, {avg_data_k['my'][-1]:.4f}, {avg_data_k['mz'][-1]:.4f})")

# Plot energy and magnetization
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(time_k, energy_k, linewidth=1.5, color='darkgreen')
axes[0].set_xlabel('Iteration', fontsize=11)
axes[0].set_ylabel('Total Energy', fontsize=11)
axes[0].set_title('Example 03 — Kagomé: Energy vs Time', fontsize=12)
axes[0].grid(True, alpha=0.3)

axes[1].plot(time_avg, avg_data_k['mx'], label='Mx', linewidth=1.5)
axes[1].plot(time_avg, avg_data_k['my'], label='My', linewidth=1.5)
axes[1].plot(time_avg, avg_data_k['mz'], label='Mz', linewidth=1.5)
axes[1].set_xlabel('Iteration', fontsize=11)
axes[1].set_ylabel('Magnetization (per atom)', fontsize=11)
axes[1].set_title('Example 03 — Kagomé: Magnetization vs Time', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("In a ~120° coplanar state, most NN pairs should have angles ≈ 120°")

# Analyze NN angles (non-collinearity signature)print("\n--- Analyzing nearest-neighbor angle distribution ---")

In [ ]:
# Analyze NN angles (non-collinearity signature)
print("\n--- Analyzing nearest-neighbor angle distribution ---")

# Load final spin configuration from restart file
restart_data_k = results_k['restart']
if restart_data_k is not None:
    # restart contains mx, my, mz arrays for final configuration
    final_spins_k = np.column_stack([
        restart_data_k['mx'].flatten(),
        restart_data_k['my'].flatten(),
        restart_data_k['mz'].flatten()
    ])
    
    print(f"Loaded {len(final_spins_k)} final spins from restart file")
    
    # Build Kagomé neighbor pairs
    # Each unit cell has 3 atoms, and we have nx_k × ny_k cells
    # Within-cell bonds: (0-1), (0-2), (1-2)
    # Between-cell bonds require neighbor indexing
    
    angles_deg = []
    
    # Collect angles from within-cell bonds (guaranteed NN)
    for cell_idx in range(nx_k * ny_k):
        base_idx = cell_idx * 3
        # Three within-cell bonds per unit cell
        for i_local, j_local in [(0, 1), (0, 2), (1, 2)]:
            i = base_idx + i_local
            j = base_idx + j_local
            if i < len(final_spins_k) and j < len(final_spins_k):
                c = float(np.dot(final_spins_k[i], final_spins_k[j]))
                c = np.clip(c, -1.0, 1.0)
                angle = np.degrees(np.arccos(c))
                angles_deg.append(angle)
    
    angles_deg = np.array(angles_deg)
    
    print(f"Computed {len(angles_deg)} NN angles")
    print(f"Mean NN angle: {angles_deg.mean():.1f}°")
    print(f"Std  NN angle: {angles_deg.std():.1f}°")
    print(f"(Expected ~120° for frustrated AFM + DMI)")
    
    # Plot histogram
    plt.figure(figsize=(8, 5))
    plt.hist(angles_deg, bins=40, edgecolor='black', alpha=0.7)
    plt.axvline(120, color='red', linestyle='--', linewidth=2, label='120° (ideal)')
    plt.xlabel("Nearest-neighbor angle (degrees)", fontsize=12)
    plt.ylabel("Count", fontsize=12)
    plt.title("Example 03 — Kagomé: NN Angle Distribution", fontsize=13)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("⚠ Restart file not found - cannot analyze final spin configuration")
    print("   Enable restart output in measurement block to analyze angles")

---

# Summary & Key Insights

## What We Learned

Using the **UppASD Python interface**, we built and ran three didactic spin-dynamics simulations:

### Example 01 — 1D Heisenberg Chain
- Simple geometry, small system size → fast convergence
- DMI + exchange compete → rich phase space
- Demonstrates energy relaxation and damping effects

### Example 02 — 2D Skyrmion Lattice
- Interfacial DMI (perpendicular to plane) + field → skyrmion stabilization
- Topological protection (non-trivial winding number)
- Shows how a single seed can grow or relax depending on field

### Example 03 — Kagomé Lattice
- **Geometric frustration** (triangular motif) + AFM exchange
- DMI selects chirality → 120° coplanar spiral
- Non-collinear magnetism (fundamental to spin glasses, multiferroics)

## Key UppASD Concepts

1. **SpinSystem**: Holds lattice, positions, species, and initial moments. Writes `posfile` and `momfile`.
2. **ExchangeShellTable & DMIShellTable**: Manage pair interactions. Write `jfile` and `dmfile` for Fortran.
3. **ASDInput**: Configures simulation parameters (dynamics, measurement, etc.). Writes `inpsd.dat`.
4. **ASDWorkspace**: Orchestrates file I/O and workspace layout.
5. **UppASDSimulator**: Calls Fortran LLG solver; Fortran is authoritative for physics.
6. **ASDResults**: Parses output (energy, magnetization, trajectories).

## Files Are the Contract

- **inpsd.dat** — Input parameters (delta-only: UppASD writes only explicit settings)
- **posfile** — Atomic positions
- **momfile** — Initial moments  
- **jfile** — Exchange interactions  
- **dmfile** — DMI interactions  
- **energy.dat**, **mag.dat** — Output (readable across CLI, GUI, notebooks)

## Next Steps

Try modifying:
- **System size** (N_chain, Nx_sq, Ny_sq) and measure scaling
- **Interaction strengths** (J, D, Bz) and explore phase transitions
- **Damping** and see how relaxation speed changes
- **Temperature** (temp ≠ 0) to include thermal fluctuations

For more advanced features, consult the reference notebooks in `notebooks/` and the UppASD documentation.